In [1]:
from pycocotools.coco import COCO
import requests

coco = COCO('datasets/COCO/annotations/instances_train2017.json')
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
# print('COCO categories: \n{}\n'.format(' '.join(nms)))





loading annotations into memory...
Done (t=28.81s)
creating index...
index created!


In [2]:
motorIds = coco.getCatIds(catNms=['motorcycle'])
motorimgIds = coco.getImgIds(catIds=motorIds)
motorimages = coco.loadImgs(motorimgIds)
print("imgIds: ", len(motorimgIds))
print("images: ", len(motorimages))

imgIds:  3502
images:  3502


In [10]:
for im in motorimages[0:1]:
#     print("im: ", im)
    img_data = requests.get(im['coco_url']).content
    with open('datasets/COCO/images/' + im['file_name'], 'wb') as handler:
        handler.write(img_data)

In [3]:
personIds = coco.getCatIds(catNms=['person'])
perosnimgIds = coco.getImgIds(catIds=personIds)
personimages = coco.loadImgs(perosnimgIds)
print("imgIds: ", len(perosnimgIds))
print("images: ", len(personimages))

personimages = personimages[0:10000]
print("images: ", len(personimages))
# print(personimages[9999])

# for im in personimages[5000:5001]:
#     img_data = requests.get("http://images.cocodataset.org/train2017/"+ im['file_name']).content
#     with open('datasets/COCO/images_helmet/Person/' + im['file_name'], 'wb') as handler:
#         handler.write(img_data)

imgIds:  64115
images:  64115
images:  10000


In [5]:
import os,json
import xml.etree.ElementTree as ET
import glob
import cv2
import numpy as np
import shutil




json_dict = {
    "info":{
        "year": "2020",
        "version": "1",
        "description": "Exported from COCO dataset",
        "contributor": "",
        "url": "http://cocodataset.org/#detection-2017",
        "date_created": "2020-04-23T04:36:14+00:00"
    },
    "license":{
            "id": 1,
            "url": "",
            "name": "Unknown"
    },
    
    "categories": [
        {
            "id": 0,
            "name": "Helmet",
            
        }, 
        {
            "id": 1,
            "name": "Motorcycle",
            
        },
        {
            "id": 2,
            "name": "Person",
            
        },
  
    ],
    
    "images": [], "type": "instances", "annotations": []}

print("Motorcycle Images",len(motorimages))
for im in motorimages:
    json_dict["images"].append(im)
    annIds = coco.getAnnIds(imgIds=im['id'], catIds=4, iscrowd=None)
    anns = coco.loadAnns(annIds)
    for i in range(len(anns)):
        annotation = {
            'segmentation':[],
             'area': anns[i]['area'],
             'iscrowd': 0,
             'image_id': anns[i]['image_id'],
             'bbox':anns[i]['bbox'],
             'category_id': 1,
             'id':anns[i]['id']
        }
        json_dict["annotations"].append(annotation)

print("Person Images",len(personimages))

for im in personimages:
    json_dict["images"].append(im)
    annIds = coco.getAnnIds(imgIds=im['id'], catIds=1, iscrowd=None)
    anns = coco.loadAnns(annIds)
    for i in range(len(anns)):
        annotation = {
            'segmentation':[],
             'area': anns[i]['area'],
             'iscrowd': 0,
             'image_id': anns[i]['image_id'],
             'bbox':anns[i]['bbox'],
             'category_id': 2,
             'id':anns[i]['id']
        }
        json_dict["annotations"].append(annotation)



txt_files = glob.glob(os.path.join('helmet_images/images/', "*.txt"))
print("Helmet Images",len(txt_files))

bnd_id = 0
img_id = 0
for txt_file in txt_files:
    with open(txt_file, "r") as tf:
        lines = tf.readlines()
        filedir = txt_file.split('/')[2].split('.')[0] + '.jpg'
        img = cv2.imread('helmet_images/images/' + filedir)
        height, width, depth = img.shape
        images_tmp = {}
        images_tmp["file_name"] = filedir
        images_tmp["height"] = height
        images_tmp["width"] = width
        images_tmp["id"] = img_id
        
        json_dict["images"].append(images_tmp)
        for i in range(len(lines)):
            tag_list = lines[i].split(' ')
            yolo_x = float(tag_list[1])
            yolo_y = float(tag_list[2])
            yolo_w = float(tag_list[3])
            yolo_h = float(tag_list[4])
            x1, y1 = int((yolo_x - yolo_w/2)*width) , int((yolo_y - yolo_h/2)*height)
            x2, y2 = int((yolo_x + yolo_w/2)*width), int(abs(yolo_y + yolo_h/2)*height)
            w = x2-x1
            h = y2-y1
            
            annotations_tmp = {}
            annotations_tmp["id"] = bnd_id
            bnd_id += 1
            annotations_tmp["image_id"] = img_id
            annotations_tmp["segmentation"] = []
            annotations_tmp["ignore"] = 0
            annotations_tmp["area"] = (x2-x1)*(y2-y1)
            annotations_tmp["iscrowd"] = 0
            annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1]
            annotations_tmp["category_id"] = 0
            json_dict["annotations"].append(annotations_tmp)
            
        img_id += 1
# print(json_dict)

print(len(json_dict['categories']))
print(len(json_dict['images']))
print(len(json_dict['annotations']))

# print(len(json_dict))

import os,json
json_file = "coco_dataset_3class/annotations/instances_Images.json"
os.makedirs(os.path.dirname(json_file), exist_ok=True)
json_fp = open(json_file, "w")
json_str = json.dumps(json_dict)
json_fp.write(json_str)
json_fp.close()



Motorcycle Images 3502
Person Images 10000
Helmet Images 11833
3
25335
74958


In [6]:
import glob,os,shutil
# files = glob.glob(os.path.join('datasets/Dataset_nl/train/Helmet/', "*.xml"))
# print(len(files))

# for i in files:
#     img_file = i.split('.')[0] + '.jpg'
#     shutil.copy(img_file,"datasets/COCO/images_helmet/Helmet/")

motor_files = []
for i in glob.glob(os.path.join('datasets/COCO/images_helmet/Motorcycle/', "*.jpg")):
    motor_files.append(i.split('/')[4])
    
person_files = []  
for i in glob.glob(os.path.join('datasets/COCO/images_helmet/Person/', "*.jpg")):
    person_files.append(i.split('/')[4])

helmet_files = []
for i in glob.glob(os.path.join('datasets/COCO/images_helmet/Helmet/', "*.jpg")):
    helmet_files.append(i.split('/')[4])

print(len(motor_files))
print(len(person_files))
# print(len(helmet_files))
# os.mkdir("coco_dataset_3class/Images")
# os.mkdir("coco_dataset_3class/Images/Person")
# os.mkdir("coco_dataset_3class/Images/Motorcycle")
# os.mkdir("coco_dataset_3class/Images/Helmet")


# cnt = 0
# for i in person_files:
#     shutil.copy("datasets/COCO/images_helmet/Person/" + i,"coco_dataset_3class/Images/")

3502
10000
